In [2]:
import os
import shutil
import cv2
import xml.etree.ElementTree as et
from sklearn.model_selection import train_test_split
!pip install ultralytics
from ultralytics import YOLO as yolo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.9/871.9 kB 18.5 MB/s eta 0:00:00


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
def normalize(crop_width, crop_height, data):
    box_width = data[2] - data[0]
    box_height = data[3] - data[1]
    x_center = (data[0] + data[2]) / 2
    y_center = (data[1] + data[3]) / 2
    x_norm = x_center / crop_width
    y_norm = y_center / crop_height
    box_width_norm = box_width / crop_width
    box_height_norm = box_height / crop_height
    return x_norm, y_norm, box_width_norm, box_height_norm

def crop_and_adjust_annotations(image_file, annotation_file, cropped_image_dir, new_label_dir, class_map):
    image_path = os.path.join(source_dir, image_file)
    img = cv2.imread(image_path)

    if img is None:
        print(f"Error reading image {image_file}")
        return

    if not os.path.exists(annotation_file):
        print(f"Annotation file {annotation_file} not found")
        return

    tree = et.parse(annotation_file)
    root = tree.getroot()

    person_id = 1
    for obj in root.findall("object"):
        name = obj.find("name").text
        bbox = obj.find("bndbox")

        if name == "person":
            xmin = int(float(bbox.find("xmin").text))
            ymin = int(float(bbox.find("ymin").text))
            xmax = int(float(bbox.find("xmax").text))
            ymax = int(float(bbox.find("ymax").text))

            new_img = img[ymin:ymax, xmin:xmax]  # Crop the person region
            new_img_filename = f"person_{person_id}_{os.path.basename(image_file)}"
            new_image_path = os.path.join(cropped_image_dir, new_img_filename)
            cv2.imwrite(new_image_path, new_img)

            print(f"Person {person_id} cropped and saved at {new_image_path}")
            new_label_filename = f"person_{person_id}_{os.path.basename(annotation_file)}"
            new_label_path = os.path.join(new_label_dir, new_label_filename.replace(".xml", ".txt"))

            # Open label file in append mode to avoid overwriting
            with open(new_label_path, 'a') as out:
                for obj_ppe in root.findall("object"):
                    ppe_name = obj_ppe.find("name").text
                    ppe_bbox = obj_ppe.find("bndbox")
                    if ppe_name in class_map:
                        ppe_xmin = int(float(ppe_bbox.find("xmin").text))
                        ppe_ymin = int(float(ppe_bbox.find("ymin").text))
                        ppe_xmax = int(float(ppe_bbox.find("xmax").text))
                        ppe_ymax = int(float(ppe_bbox.find("ymax").text))

                        # Check if the PPE bounding box lies within the person bounding box
                        if ppe_xmin >= xmin and ppe_xmax <= xmax and ppe_ymin >= ymin and ppe_ymax <= ymax:
                            adjusted_xmin = ppe_xmin - xmin
                            adjusted_ymin = ppe_ymin - ymin
                            adjusted_xmax = ppe_xmax - xmin
                            adjusted_ymax = ppe_ymax - ymin

                            # Normalize the bounding box to the cropped person image
                            data = (adjusted_xmin, adjusted_ymin, adjusted_xmax, adjusted_ymax)
                            new_data = normalize(xmax - xmin, ymax - ymin, data)

                            # Write normalized data to the label file
                            out.write(f"{class_map[ppe_name]} " + " ".join(f"{a:.6f}" for a in new_data) + "\n")

            person_id += 1



if __name__ == "__main__":
    source_dir = "/content/drive/MyDrive/Colab Notebooks/Internship Works/ppe_datasets/images"
    label_dir = "/content/drive/MyDrive/Colab Notebooks/Internship Works/ppe_datasets/labels"
    cropped_image_dir = "/content/drive/MyDrive/Colab Notebooks/Internship Works/ppe_datasets/cropped/images"
    new_label_dir = "/content/drive/MyDrive/Colab Notebooks/Internship Works/ppe_datasets/cropped/labels"

    os.makedirs(cropped_image_dir,exist_ok=True)
    os.makedirs(new_label_dir,exist_ok=True)

    class_ppe = "/content/drive/MyDrive/Colab Notebooks/Internship Works/ppe_datasets/classes_ppe.txt"
    class_map = {}
    with open(class_ppe,'r') as file:
        for id, line in enumerate(file):
            class_map[line.strip()] = id

    for image_file in os.listdir(source_dir):
        annotation_file = os.path.join(label_dir,image_file.replace(".jpg",".xml"))
        crop_and_adjust_annotations(image_file, annotation_file, cropped_image_dir, new_label_dir, class_map)


Person 1 cropped and saved at /content/drive/MyDrive/Colab Notebooks/Internship Works/ppe_datasets/cropped/images/person_1_Aitin3205_jpg.rf.0e0bc650138d83144f9e7196dd2bec38.jpg
Person 2 cropped and saved at /content/drive/MyDrive/Colab Notebooks/Internship Works/ppe_datasets/cropped/images/person_2_Aitin3205_jpg.rf.0e0bc650138d83144f9e7196dd2bec38.jpg
Person 3 cropped and saved at /content/drive/MyDrive/Colab Notebooks/Internship Works/ppe_datasets/cropped/images/person_3_Aitin3205_jpg.rf.0e0bc650138d83144f9e7196dd2bec38.jpg
Person 4 cropped and saved at /content/drive/MyDrive/Colab Notebooks/Internship Works/ppe_datasets/cropped/images/person_4_Aitin3205_jpg.rf.0e0bc650138d83144f9e7196dd2bec38.jpg
Person 5 cropped and saved at /content/drive/MyDrive/Colab Notebooks/Internship Works/ppe_datasets/cropped/images/person_5_Aitin3205_jpg.rf.0e0bc650138d83144f9e7196dd2bec38.jpg
Person 1 cropped and saved at /content/drive/MyDrive/Colab Notebooks/Internship Works/ppe_datasets/cropped/images/p

In [21]:
def count_files(directory):
    # Get a list of all items (files and directories) in the specified directory
    all_items = os.listdir(directory)

    # Filter only the files
    files = [f for f in all_items if os.path.isfile(os.path.join(directory, f))]

    return len(files)

print(count_files(cropped_image_dir))
print(count_files(new_label_dir))

1284
1284


In [22]:
train_dir = "/content/drive/MyDrive/Colab Notebooks/Internship Works/ppe_datasets/train"
val_dir = "/content/drive/MyDrive/Colab Notebooks/Internship Works/ppe_datasets/val"
test_dir = "/content/drive/MyDrive/Colab Notebooks/Internship Works/ppe_datasets/test"

def dest(dest_dir):
    subdirect = ["images","labels"]
    for split in subdirect:
        full_path = os.path.join(dest_dir,split)
        if not os.path.exists(full_path):
            os.makedirs(full_path)
            print("Directories created")
        else:
            print("Directories already created")

#Source directory
images_dir = os.listdir(cropped_image_dir)
labels_dir = os.listdir(new_label_dir)

trainval_images, test_images, trainval_labels, test_labels = train_test_split(images_dir, labels_dir, test_size=0.2, random_state=42)
train_images, val_images, train_labels, val_labels = train_test_split(trainval_images, trainval_labels, test_size=0.25, random_state=42)

#Move the data
def move(files,source_dir,out_dir, subdir):
    dest_path = os.path.join(out_dir,subdir)
    for file in files:
        source_file = os.path.join(source_dir,file)
        destin_file = os.path.join(dest_path,file)
        os.rename(source_file,destin_file)
        print("moved")

dest(train_dir)
dest(val_dir)
dest(test_dir)

move(train_images,cropped_image_dir,train_dir,"images")
move(train_labels,new_label_dir,train_dir,"labels")
move(val_images,cropped_image_dir,val_dir,"images")
move(val_labels,new_label_dir,val_dir,"labels")
move(test_images,cropped_image_dir,test_dir,"images")
move(test_labels,new_label_dir,test_dir,"labels")

Directories created
Directories created
Directories created
Directories created
Directories created
Directories created
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
moved
move

In [ ]:
model = yolo("yolov8n.yaml")
model = yolo("yolov8n.pt")
model = yolo("yolov8n.yaml").load("yolov8n.pt")

results = model.train(data="/content/drive/MyDrive/Colab Notebooks/Internship Works/scripts/ppe.yaml", epochs=10, imgsz=640)

Transferred 355/355 items from pretrained weights
Ultralytics YOLOv8.2.92 🚀 Python-3.10.12 torch-2.4.0+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/drive/MyDrive/Colab Notebooks/Internship Works/scripts/ppe.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=yolov8n.pt, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fr

train: Scanning /content/drive/MyDrive/Colab Notebooks/Internship Works/ppe_datasets/train/labels... 767 images, 123 backgrounds, 3 corrupt: 100%|██████████| 770/770 [00:16<00:00, 46.89it/s]

train: WARNING ⚠️ /content/drive/MyDrive/Colab Notebooks/Internship Works/ppe_datasets/train/images/person_10_001305.jpg: ignoring corrupt image/label: image size (21, 7) <10 pixels
train: WARNING ⚠️ /content/drive/MyDrive/Colab Notebooks/Internship Works/ppe_datasets/train/images/person_5_001790.jpg: ignoring corrupt image/label: image size (18, 7) <10 pixels
train: WARNING ⚠️ /content/drive/MyDrive/Colab Notebooks/Internship Works/ppe_datasets/train/images/person_6_001790.jpg: ignoring corrupt image/label: image size (16, 7) <10 pixels


train: New cache created: /content/drive/MyDrive/Colab Notebooks/Internship Works/ppe_datasets/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/drive/MyDrive/Colab Notebooks/Internship Works/ppe_datasets/val/labels... 257 images, 37 backgrounds, 0 corrupt: 100%|██████████| 257/257 [00:05<00:00, 50.29it/s]


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      10/10         0G      1.114      1.861       1.22         27        640: 100%|██████████| 48/48 [12:18<00:00, 15.39s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:50<00:00,  5.61s/it]

                   all        257        492      0.596      0.445      0.411      0.261



10 epochs completed in 2.195 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.92 🚀 Python-3.10.12 torch-2.4.0+cu121 CPU (Intel Xeon 2.20GHz)
YOLOv8n summary (fused): 168 layers, 3,007,403 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  44%|████▍     | 4/9 [00:30<00:34,  6.88s/it]

In [7]:
weigh = "/content/drive/MyDrive/Colab Notebooks/Internship Works/weights/ppe_detection.pt"
model = yolo(weigh)
results = model.predict("/content/drive/MyDrive/Colab Notebooks/Internship Works/ppe_datasets/test/images/person_1_001060.jpg",save=True)


image 1/1 /content/drive/MyDrive/Colab Notebooks/Internship Works/ppe_datasets/test/images/person_1_001060.jpg: 640x320 1 hard-hat, 1 boots, 287.4ms
Speed: 13.1ms preprocess, 287.4ms inference, 27.0ms postprocess per image at shape (1, 3, 640, 320)
Results saved to runs/detect/predict


In [8]:
predicted_bboxes = []
for result in results:
    for box in result.boxes:
        x_min, y_min, x_max, y_max = box.xyxy[0].tolist()  # Bounding box coordinates
        confidence = box.conf[0].item()                    # Confidence score
        label = int(box.cls[0].item())                     # Class label
        predicted_bboxes.append((label, x_min, y_min, x_max, y_max, confidence))
print(predicted_bboxes)

[(0, 12.125648498535156, 0.0, 56.050148010253906, 38.078521728515625, 0.8300375938415527), (4, 55.88288497924805, 233.68191528320312, 88.47761535644531, 249.6482391357422, 0.5699059367179871)]
